In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# !rm -rf "/content/drive/MyDrive/Colab Notebooks/tin_intro/data/tiny-imagenet-200"

In [4]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/tin_intro')

In [5]:
!pip install einops
!pip install thop

In [6]:
import time

import PIL
import torch
import torchvision
import torch.nn.functional as F
from torch import nn
import torch.nn.init as init

from einops import rearrange
from common import *
from ViTResNet18 import *
from TinyImageNet import TinyImageNet

In [ ]:
# !wget -P "/content/drive/MyDrive/Colab Notebooks/tin_intro/data/tiny-imagenet-200" http://cs231n.stanford.edu/tiny-imagenet-200.zip

In [ ]:

#!unzip "/content/drive/MyDrive/Colab Notebooks/tin_intro/data/tiny-imagenet-200.zip" -d "/content/drive/MyDrive/Colab Notebooks/tin_intro/data/"
# !unzip path_to_file.zip -d path_to_directoryA


Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/Colab Notebooks/tin_intro/data/tiny-imagenet-200/val/images/val_3979.JPEG  
  inflating: /content/drive/MyDrive/Colab Notebooks/tin_intro/data/tiny-imagenet-200/val/images/val_3963.JPEG  
  inflating: /content/drive/MyDrive/Colab Notebooks/tin_intro/data/tiny-imagenet-200/val/images/val_7199.JPEG  
  inflating: /content/drive/MyDrive/Colab Notebooks/tin_intro/data/tiny-imagenet-200/val/images/val_2752.JPEG  
  inflating: /content/drive/MyDrive/Colab Notebooks/tin_intro/data/tiny-imagenet-200/val/images/val_9687.JPEG  
  inflating: /content/drive/MyDrive/Colab Notebooks/tin_intro/data/tiny-imagenet-200/val/images/val_9407.JPEG  
  inflating: /content/drive/MyDrive/Colab Notebooks/tin_intro/data/tiny-imagenet-200/val/images/val_3603.JPEG  
  inflating: /content/drive/MyDrive/Colab Notebooks/tin_intro/data/tiny-imagenet-200/val/images/val_3412.JPEG  
  inflating: /content/drive/MyDrive/Colab Notebooks/t

In [ ]:
PATH_TO_IMAGE_NET = "/content/drive/MyDrive/Colab Notebooks/tin_intro/data/tiny-imagenet-200"
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_VAL = 100
device = torch.device("cuda:0")
root_path = "/content/drive/MyDrive/Colab Notebooks/tin_intro/"

transform_train = torchvision.transforms.Compose(
     [torchvision.transforms.RandomHorizontalFlip(),
     torchvision.transforms.RandomRotation(10, resample=PIL.Image.BILINEAR),
     torchvision.transforms.RandomAffine(8, translate=(.15,.15)),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


transform_val = torchvision.transforms.Compose([
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = TinyImageNet(PATH_TO_IMAGE_NET, split='train', transform=transform_train, in_memory=False)
val_dataset = TinyImageNet(PATH_TO_IMAGE_NET, split='val', transform=transform_val, in_memory=False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE_VAL, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1201: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


In [ ]:

model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=False).to(device)
EPOCHS = 90
check_on_dataset(model, train_loader, val_loader, EPOCHS, "TinyImageNet", "resnet18", root_path=root_path)


Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to /root/.cache/torch/hub/v0.9.0.zip


Epoch: 1
[    0/100000 (  0%)]  Loss: 7.1029
[10000/100000 ( 10%)]  Loss: 5.2000
[20000/100000 ( 20%)]  Loss: 5.0014
[30000/100000 ( 30%)]  Loss: 4.9337
[40000/100000 ( 40%)]  Loss: 4.6268
[50000/100000 ( 50%)]  Loss: 4.3630
[60000/100000 ( 60%)]  Loss: 4.4490
[70000/100000 ( 70%)]  Loss: 4.2609
[80000/100000 ( 80%)]  Loss: 4.3910
[90000/100000 ( 90%)]  Loss: 4.2947
Execution time: 188.78 seconds
Epoch: 2
[    0/100000 (  0%)]  Loss: 4.0690
[10000/100000 ( 10%)]  Loss: 4.0613
[20000/100000 ( 20%)]  Loss: 4.2879
[30000/100000 ( 30%)]  Loss: 4.0226
[40000/100000 ( 40%)]  Loss: 3.8616
[50000/100000 ( 50%)]  Loss: 3.8541
[60000/100000 ( 60%)]  Loss: 3.4211
[70000/100000 ( 70%)]  Loss: 3.8699
[80000/100000 ( 80%)]  Loss: 3.8853
[90000/100000 ( 90%)]  Loss: 3.6402
Execution time: 187.47 seconds
Epoch: 3
[    0/100000 (  0%)]  Loss: 3.5346
[10000/100000 ( 10%)]  Loss: 3.3751
[20000/100000 ( 20%)]  Loss: 3.3321
[30000/100000 ( 30%)]  Loss: 3.5013
[40000/100000 ( 40%)]  Loss: 3.2836
[50000/1000

In [ ]:
from thop import profile
data = torch.randn(1, 3, 224, 224)
flops, params = profile(model, inputs=(data.to(device), ))
print(f'flops = {flops / 1e6}FLOPs(M)')
print(f'params = {params / 1e6}Params(M)')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.BasicBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.ResNet'>. Treat it as zero Macs and zero Params.
flops = 1819.066368FLOPs(M)
params = 11.689512Params(M)


In [ ]:
model = ViTResNet18(BasicBlock, [2, 2, 2], BATCH_SIZE_TRAIN, num_classes=200, num_tokens=32, tokenizers_type=['filter', 'filter']).to(device)
EPOCHS = 90
check_on_dataset(model, train_loader, val_loader, EPOCHS, "TinyImageNet", "ViTResNet18(tokens=32)", root_path=root_path)

Epoch: 1
[    0/100000 (  0%)]  Loss: 5.5952
[10000/100000 ( 10%)]  Loss: 5.1370
[20000/100000 ( 20%)]  Loss: 4.9140
[30000/100000 ( 30%)]  Loss: 4.8386
[40000/100000 ( 40%)]  Loss: 4.7145
[50000/100000 ( 50%)]  Loss: 4.6100
[60000/100000 ( 60%)]  Loss: 4.6309
[70000/100000 ( 70%)]  Loss: 4.6066
[80000/100000 ( 80%)]  Loss: 4.2118
[90000/100000 ( 90%)]  Loss: 4.4977
Execution time: 187.94 seconds
Epoch: 2
[    0/100000 (  0%)]  Loss: 4.0941
[10000/100000 ( 10%)]  Loss: 4.2281
[20000/100000 ( 20%)]  Loss: 4.1292
[30000/100000 ( 30%)]  Loss: 4.2263
[40000/100000 ( 40%)]  Loss: 4.0533
[50000/100000 ( 50%)]  Loss: 4.1942
[60000/100000 ( 60%)]  Loss: 3.9668
[70000/100000 ( 70%)]  Loss: 4.4682
[80000/100000 ( 80%)]  Loss: 3.9712
[90000/100000 ( 90%)]  Loss: 3.9306
Execution time: 186.54 seconds
Epoch: 3
[    0/100000 (  0%)]  Loss: 4.0472
[10000/100000 ( 10%)]  Loss: 3.6610
[20000/100000 ( 20%)]  Loss: 3.8976
[30000/100000 ( 30%)]  Loss: 3.6764
[40000/100000 ( 40%)]  Loss: 3.4693
[50000/1000

In [ ]:
# Все тесты сверху были на 64x64 и плохо показали себя. Теперь запущу те же тесты на 224x224

In [ ]:
%pwd

'/content/drive/MyDrive'

In [ ]:
%cd Colab\ Notebooks/tin_intro/

/content/drive/MyDrive/Colab Notebooks/tin_intro


In [ ]:
!python to224.py

In [7]:
PATH_TO_IMAGE_NET = "/content/drive/MyDrive/Colab Notebooks/tin_intro/data/tiny-imagenet-200"
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_VAL = 100
device = torch.device("cuda:0")
root_path = "/content/drive/MyDrive/Colab Notebooks/tin_intro/"

transform_train = torchvision.transforms.Compose(
     [torchvision.transforms.RandomHorizontalFlip(),
     torchvision.transforms.RandomRotation(10, resample=PIL.Image.BILINEAR),
     torchvision.transforms.RandomAffine(8, translate=(.15,.15)),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


transform_val = torchvision.transforms.Compose([
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = TinyImageNet(PATH_TO_IMAGE_NET, split='train', transform=transform_train, in_memory=False)
val_dataset = TinyImageNet(PATH_TO_IMAGE_NET, split='val', transform=transform_val, in_memory=False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE_VAL, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1201: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


In [8]:
for data, target in train_loader:
  print(data.shape)
  break

torch.Size([100, 3, 224, 224])


In [9]:
# 
model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=False).to(device)
EPOCHS = 90
check_on_dataset(model, train_loader, val_loader, EPOCHS, "TinyImageNet", "resnet18_224x224", root_path=root_path)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Epoch: 1
[    0/100000 (  0%)]  Loss: 7.0140
[10000/100000 ( 10%)]  Loss: 4.9844
[20000/100000 ( 20%)]  Loss: 4.8690
[30000/100000 ( 30%)]  Loss: 4.3517
[40000/100000 ( 40%)]  Loss: 4.4928
[50000/100000 ( 50%)]  Loss: 4.3062
[60000/100000 ( 60%)]  Loss: 4.3608
[70000/100000 ( 70%)]  Loss: 4.4081
[80000/100000 ( 80%)]  Loss: 3.8273
[90000/100000 ( 90%)]  Loss: 3.9068
Execution time: 738.42 seconds
Epoch: 2
[    0/100000 (  0%)]  Loss: 3.6279
[10000/100000 ( 10%)]  Loss: 3.8526
[20000/100000 ( 20%)]  Loss: 3.6005
[30000/100000 ( 30%)]  Loss: 3.6384
[40000/100000 ( 40%)]  Loss: 3.3635
[50000/100000 ( 50%)]  Loss: 3.4972
[60000/100000 ( 60%)]  Loss: 3.4906
[70000/100000 ( 70%)]  Loss: 3.4174
[80000/100000 ( 80%)]  Loss: 3.2818
[90000/100000 ( 90%)]  Loss: 2.8529
Execution time: 693.96 seconds
Epoch: 3
[    0/100000 (  0%)]  Loss: 3.4510
[10000/100000 ( 10%)]  Loss: 2.6368
[20000/100000 ( 20%)]  Loss: 3.2787
[30000/100000 ( 30%)]  Loss: 3.2871
[40000/100000 ( 40%)]  Loss: 3.0070
[50000/1000

KeyboardInterrupt: ignored

In [ ]:
model = ViTResNet18(BasicBlock, [2, 2, 2], BATCH_SIZE_TRAIN, num_classes=200, num_tokens=32, tokenizers_type=['filter', 'filter']).to(device)
EPOCHS = 90
check_on_dataset(model, train_loader, val_loader, EPOCHS, "TinyImageNet", "ViTResNet18(tokens=32)(224x224)", root_path=root_path)